In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pyperclip
import time

driver = webdriver.Chrome()
driver.get("https://login.microsoftonline.com/")
time.sleep(1)

In [98]:
pyperclip.copy("아이디")
id_input = driver.find_element(By.CSS_SELECTOR, "#i0116")
id_input.clear()
id_input.send_keys(Keys.COMMAND, 'v', Keys.ENTER)
time.sleep(1)

pyperclip.copy("비밀번호")
pw_input = driver.find_element(By.CSS_SELECTOR, "#i0118")
pw_input.clear()
pw_input.send_keys(Keys.COMMAND, 'v', Keys.ENTER)
time.sleep(1)

driver.find_element(By.CSS_SELECTOR, "#idSIButton9").send_keys(Keys.ENTER)
time.sleep(1)


In [99]:
driver.get("https://gw.plateer.com/")
time.sleep(1)

driver.find_element(By.CSS_SELECTOR, "#tilesHolder > div.tile-container > div > div.table").click()
time.sleep(1)

In [59]:
import os
import sys
import sqlite3

"""현재 실행 파일의 디렉토리에 SQLite 데이터베이스 파일 경로 반환"""
if getattr(sys, 'frozen', False):  # PyInstaller로 빌드된 경우
    app_path = os.path.dirname(sys.executable)
elif '__file__' in globals():  # 스크립트로 실행되는 경우
    app_path = os.path.dirname(os.path.abspath(__file__))
else:
    app_path = os.getcwd()

db_path = os.path.join(app_path, 'data.db')



"""SQLite 데이터베이스 초기화"""
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 테이블이 없는 경우 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS data (
    no INTEGER PRIMARY KEY AUTOINCREMENT,
    id TEXT UNIQUE NOT NULL,
    name TEXT NOT NULL,
    birthday TEXT NOT NULL,
    department TEXT,
    image TEXT
)
''')

conn.commit()
conn.close()

In [109]:
"""SQLite 데이터베이스에서 저장된 데이터 가져오기"""
search = '검색어'
where_name = (f'%{search}%', ) 

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute('SELECT * FROM data WHERE name like ?', where_name)
rows = cursor.fetchall()
conn.close()

if rows:
    print("Stored Data:")
    for row in rows:
        print(f"No: {row[0]}, Name: {row[2]}, ID: {row[1]}, Department: {row[4]}, BirthDay: {row[3]},  Image: {row[5]}")
else:
    print("No data found in the database.")

Stored Data:
No: 68, Name: 김지은, ID: ccomo7071, Department: 프론트개발파트, BirthDay: 1999-03-14,  Image: /ezCommon/downloadAttach.do?&filePath=/fileroot/0/files/upload_personal/photo/ccomo7071.png
No: 139, Name: 이지은, ID: jieun_0622, Department: 영업팀, BirthDay: 1993-06-22,  Image: /ezCommon/downloadAttach.do?&filePath=/fileroot/0/files/upload_personal/photo/jieun_0622.png
No: 202, Name: 이지은, ID: jieun.lee, Department: EC솔루션개발팀, BirthDay: 1992-09-11,  Image: /ezCommon/downloadAttach.do?&filePath=/fileroot/0/files/upload_personal/photo/jieun.lee.png
No: 214, Name: 최지은, ID: jiunny, Department: 데이터컨설팅팀, BirthDay: 1997-09-21,  Image: /ezCommon/downloadAttach.do?&filePath=/fileroot/0/files/upload_personal/photo/jiunny.png
No: 254, Name: 손지은, ID: je.son, Department: 마케팅팀, BirthDay: 1990-11-04,  Image: /ezCommon/downloadAttach.do?&filePath=/fileroot/0/files/upload_personal/photo/je.son.png
No: 266, Name: 백지은, ID: brie, Department: 고객성공팀, BirthDay: 1997-11-15,  Image: /ezCommon/downloadAttach.do?&filePa

In [108]:
import requests

loginCookie = driver.get_cookie('loginCookie').get("value")
header = {
    "Cookie": f"loginCookie={loginCookie}"
}

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

data = []
cursor.execute('DELETE FROM data')
cursor.execute('DELETE FROM sqlite_sequence WHERE name="data"')

for month in range( 1, 13 ):
    response = requests.get(f"https://gw.plateer.com/ezNewPortal/getMonthlyBirthdayEmployees.do?birthdayMonth={month}&birthdayCurPage=1&birthdayCount=100", headers=header)
    birthday_list = response.json()['birthdayList']
    db_data_info = [(user["userId"], user["userName"], user["userBirthday"], user["userDeptName"], user["userImg"]) for user in birthday_list]
    
    data = [*data, *db_data_info]
    
cursor.executemany('INSERT INTO data (id, name, birthday, department, image) VALUES (?, ?, ?, ?, ?)', data)
conn.commit()
conn.close()


In [110]:
driver.quit()